In [1]:
!pip install remotezip tqdm opencv-python

In [1]:
import tqdm
import random
import pathlib
import itertools

import cv2
import numpy as np
import remotezip as rz

import tensorflow as tf
import keras
from keras import layers

In [2]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
def get_max_n_frames():
    nonv = "/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence"
    v = "/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence"
    nonv_files = [nonv+"/"+f for f in os.listdir(nonv)]
    v_files = [v+"/"+f for f in os.listdir(v)]
    max_frame=0
    removed_file = []
    for cls in [nonv_files,v_files]:
        for f in cls:
            cap = cv2.VideoCapture(f)
            n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
            if n_frames>1000:
                removed_file.append(f)
            else:
                max_frame = max(max_frame,n_frames)
    print(max_frame)
    print(removed_file)
get_max_n_frames()

283
['/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/NV_992.mp4', '/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_789.mp4', '/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/V_398.mp4']


In [4]:
os.mkdir('data')
os.mkdir('data/train')
os.mkdir('data/test')
os.mkdir('data/val')
classes = os.listdir("/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset")
num_classes = len(classes)

data_videos = pd.DataFrame(columns=['class', 'path'])

for c in classes:
    os.mkdir(f"data/train/{c}")
    os.mkdir(f"data/test/{c}")
    os.mkdir(f"data/val/{c}")
    files = os.listdir(f"/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/{c}")
    if c == "NonViolence":
        files.remove("NV_992.mp4")
    else:
        files.remove("V_789.mp4")
        files.remove("V_398.mp4")
    temp =  pd.DataFrame([files,[c]*len(files)]).T
    temp.columns = ['path', 'class']
    data_videos = pd.concat([data_videos, temp])

data_videos = data_videos.sample(frac=1).reset_index(drop=True)
data_videos

,class,path
0,NonViolence,NV_984.mp4
1,Violence,V_374.mp4
2,NonViolence,NV_181.mp4
3,NonViolence,NV_982.mp4
4,Violence,V_58.mp4
...,...,...
1992,Violence,V_751.mp4
1993,NonViolence,NV_979.mp4
1994,Violence,V_494.mp4
1995,NonViolence,NV_874.avi


In [5]:
train,test = train_test_split(data_videos, test_size=0.1,random_state=42)
train,val = train_test_split(train, test_size=0.1,random_state=42)

In [6]:
def move_files(subset,data):
    if not subset in ["train","test","val"]:
        return
    else:
        if subset == 'train':
            destination = "data/train"
        elif subset == "test":
            destination = "data/test"
        elif subset == "val":
            destination = "data/val"
        for i in range(len(data)):
            path = f"{data.iloc[i]['class']}/{data.iloc[i]['path']}"
            shutil.copy(f"/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset/{path}",f"{destination}/{path}")

move_files("train",train)
move_files("test",test)
move_files("val",val)


In [7]:
def process_frame(frame):
    frame = tf.image.convert_image_dtype(frame, tf.float32)
    frame = tf.image.resize(frame, size=(122, 122))
    frame = frame/255.0
    return frame

def read_frames_from_video(video_path):
    src = cv2.VideoCapture(video_path)
    length = int(src.get(cv2.CAP_PROP_FRAME_COUNT))
    start = 283 - length
    success = 1

    frames = [np.zeros((122, 122,3)) for i in range(start)]

    while success:
        success, frame = src.read()
        if frame is not None:
            frame = process_frame(frame)
            frames.append(frame)
    src.release()
    frames = np.array(frames)
    return frames

def video_frame_generator(files, labels, batch_size=32):
    images = []
    labels_list = []
    data = zip(files, labels)

    for f, label in data:
        frames = read_frames_from_video(f)

        images.append(frames)
        labels_list.append(label)

        # Check if we have enough images for a batch
        if len(images) == batch_size:
            yield np.array(images), np.array(labels_list)
            # Reset the lists for the next batch
            images = []
            labels_list = []

    # Yield any remaining frames that didn't fill a complete batch
    if images:
        yield np.array(images), np.array(labels_list)


def getvideo(subset,data):
    path = f"data/{subset}"
    files = (data["class"]+"/"+data["path"]).tolist()
    labels = (train["class"].astype("category").cat.codes).tolist()
    return video_frame_generator(files,labels)
    


In [10]:
import itertools

train_imgs = itertools.cycle(getvideo("train",train))
test_imgs = itertools.cycle(getvideo("test",test))
val_imgs = itertools.cycle(getvideo("val",val))

In [20]:
for image,label in train_imgs:
    print(image.shape)
    break

(32, 283, 122, 122, 3)


In [12]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, Bidirectional
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

In [13]:
!git clone https://github.com/tensorflow/models.git

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Cloning into 'models'...
remote: Enumerating objects: 98359, done.
remote: Counting objects: 100% (866/866), done.
remote: Compressing objects: 100% (467/467), done.
remote: Total 98359 (delta 461), reused 717 (delta 376), pack-reused 97493 (from 1)
Receiving objects: 100% (98359/98359), 621.96 MiB | 37.46 MiB/s, done.
Resolving deltas: 100% (71454/71454), done.


In [13]:
import sys
sys.path.append("/kaggle/working/models")

In [15]:
!pip install gin-config==0.1.1
!pip install pycocotools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.3/40.3 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gin-config: filename=gin_config-0.1.1-py3-none-any.whl size=38309 sha256=0d4cf10f39a2fb18e376e3df5bf4873b2e3de21e2d1df0a69e99f861ff8bcd2d
  Stored in directory: /root/.cache/pip/wheels/aa/ad/a7/b085ddc60427ebf0d52e1643eb4906911866690c004adbcc9d
Successfully built gin-config
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.8/427.8 kB 12.6 MB/s eta 0:00:00


In [14]:
from official.projects.movinet.modeling import movinet_model
from official.projects.movinet.modeling import movinet

In [15]:
model_id = 'a0'
backbone = movinet.Movinet(model_id=model_id)
backbone.trainable = False

def build_classifier(batch_size, num_frames, resolution, backbone, num_classes):
    model = movinet_model.MovinetClassifier(backbone=backbone,num_classes=num_classes)
    model.build([batch_size, num_frames, resolution, resolution, 3])
    
    return model

In [16]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, BatchNormalization

In [17]:
model = build_classifier(32, 283, 122, backbone, 2)

In [18]:
num_epochs = 100
loss_obj = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(loss=loss_obj, optimizer="adam", metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
model.fit(train_imgs,validation_data=test_imgs,epochs=20)

Epoch 1/20


I0000 00:00:1730610562.186821   14368 service.cc:145] XLA service 0x7cf038016600 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1730610562.186912   14368 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1730610562.186918   14368 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1730610565.076074   14368 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-11-03 05:09:31.712000: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng28{k2=2,k3=0} for conv (f32[32,24,283,31,31]{4,3,2,1,0}, u8[0]{0}) custom-call(f32[32,24,283,61,61]{4,3,2,1,0}, f32[24,1,1,5,5]{4,3,2,1,0}), window={size=1x5x5 stride=1x2x2 pad=0_0x2_2x2_2}, dim_labels=bf012_oi012->bf012, feature_group_count=24, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait

      6/Unknown - 112s 5s/step - loss: 0.6920 - accuracy: 0.5260